In [1]:
# !python -m pip install torch datasets evaluate transformers

In [4]:
import datasets
import evaluate
import transformers

import numpy as np
import torch
import torchvision

c:\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_checkpoint = "google/vit-base-patch16-224-in21k" # pre-trained model from which to fine-tune
batch_size = 8 # batch size for training and evaluation

In [4]:
dataset = datasets.load_dataset("imagefolder", data_dir="D:/my_gestures", cache_dir="D:/.cache/")
dataset.cleanup_cache_files()
print(dataset)

Resolving data files: 100%|██████████| 1500/1500 [00:00<00:00, 38468.09it/s]
Found cached dataset imagefolder (D:/.cache/imagefolder/default-3512ca76eea585d5/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)
100%|██████████| 2/2 [00:00<00:00, 25.64it/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 6000
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 1500
    })
})


In [5]:
metric = evaluate.load("accuracy")

In [6]:
labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

print(labels)

['00_fist', '01_one', '02_two', '03_three', '04_four', '05_five']


In [7]:
feature_extractor = transformers.AutoFeatureExtractor.from_pretrained(model_checkpoint)
print(feature_extractor)

ViTFeatureExtractor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTFeatureExtractor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}



c:\Python38\lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [8]:
import albumentations as A

size = 224

train_transforms = A.Compose([
    A.Resize(height=size, width=size),
    A.Normalize()
])

val_transforms = A.Compose([
    A.Resize(height=size, width=size),
    A.Normalize()
])

def preprocess_train(examples):
    examples['pixel_values'] = [
        train_transforms(image=np.array(image))["image"] for image in examples["image"]
    ]
    return examples

def preprocess_val(examples):
    examples['pixel_values'] = [
        val_transforms(image=np.array(image))["image"] for image in examples["image"]
    ]
    return examples

In [9]:
train_ds = dataset['train']
val_ds = dataset['validation']

train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)

In [10]:
model = transformers.AutoModelForImageClassification.from_pretrained(
    model_checkpoint,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,
)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
model_name = model_checkpoint.split("/")[-1]

args = transformers.TrainingArguments(
    f"{model_name}-finetuned-gestures",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-3,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

In [12]:
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [13]:
def collate_fn(examples):
    images = []
    labels = []
    for example in examples:
        image = np.moveaxis(example["pixel_values"], source=2, destination=0)
        images.append(torch.from_numpy(image))
        labels.append(example["label"])
        
    pixel_values = torch.stack(images)
    labels = torch.tensor(labels)
    return {"pixel_values": pixel_values, "labels": labels}

In [14]:
trainer = transformers.Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

In [15]:
train_results = trainer.train()
# rest is optional but nice to have
trainer.save_model("D:/saved_models/vit-base-patch16-224-in21k-finetuned-gestures/")
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()


c:\Python38\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  1%|          | 10/935 [00:21<30:37,  1.99s/it] 

{'loss': 1.7832, 'learning_rate': 0.00010638297872340425, 'epoch': 0.05}


  2%|▏         | 20/935 [00:41<30:07,  1.97s/it]

{'loss': 1.5243, 'learning_rate': 0.0002127659574468085, 'epoch': 0.11}


  3%|▎         | 30/935 [01:01<29:55,  1.98s/it]

...

{'loss': 0.0162, 'learning_rate': 5.9453032104637345e-06, 'epoch': 4.96}


                                                 
100%|██████████| 935/935 [39:56<00:00,  2.84s/it]

{'eval_loss': 0.048720479011535645, 'eval_accuracy': 0.9846666666666667, 'eval_runtime': 44.7587, 'eval_samples_per_second': 33.513, 'eval_steps_per_second': 4.2, 'epoch': 4.99}


100%|██████████| 935/935 [40:00<00:00,  2.57s/it]


{'train_runtime': 2400.2691, 'train_samples_per_second': 12.499, 'train_steps_per_second': 0.39, 'train_loss': 0.33161284952240194, 'epoch': 4.99}
***** train metrics *****
  epoch                    =       4.99
  train_loss               =     0.3316
  train_runtime            = 0:40:00.26
  train_samples_per_second =     12.499
  train_steps_per_second   =       0.39


In [16]:
metrics = trainer.evaluate()
# some nice to haves:
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

100%|██████████| 188/188 [00:45<00:00,  4.14it/s]

***** eval metrics *****
  epoch                   =       4.99
  eval_accuracy           =     0.9847
  eval_loss               =     0.0487
  eval_runtime            = 0:00:45.67
  eval_samples_per_second =     32.843
  eval_steps_per_second   =      4.116
